# OCR

### Подготовка

In [1]:
colab = False

In [ ]:
if colab:
    !pip install requests
    !pip install random_word
    !pip install tqdm
    !pip install streamlit>=1.0.0
    !pip install wandb>=0.10.31
    !pip install matplotlib>=3.1.0
    !pip install defusedxml
    !pip install opencv-python-headless
    !pip install anyascii
    !pip install ipykernel==6.29.5
    !pip install pandas
    !pip install wget
    !pip install pyyaml
    !pip install natsort

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import random
from PIL import Image, ImageDraw, ImageFont
from random_word import RandomWords
import string
import shutil
import wget
import zipfile
import yaml
import wget
import subprocess

#### Загружаем датасет с Hugging Face и распаковываем

Если нужно перегенерировать датасет, в ячейке ниже должно быть True. В таком случае загружаем кириллический датасет с HF и дополняем его своей синтетикой. Иначе загружаем готовый датасет со всеми необходимыми символами

In [ ]:
generate_dataset = True

In [ ]:
if generate_dataset:
  file_name = 'data_1.zip'
  if not os.path.exists(file_name):
    wget.download('https://huggingface.co/datasets/DonkeySmall/OCR-Cyrillic-Printed-1/resolve/main/data_1.zip')
  with zipfile.ZipFile(file_name, 'r') as zip_ref:
    for file in tqdm(zip_ref.infolist(), desc='Extracting files'):
        zip_ref.extract(file, 'dataset')
else:
   # клонируем с HF
  wget.download('https://huggingface.co/datasets/smthrgnl/ocr_cyrillic_english/resolve/main/images.zip')
  wget.download('https://huggingface.co/datasets/smthrgnl/ocr_cyrillic_english/resolve/main/labels.csv')
  with zipfile.ZipFile('images.zip') as zip_ref:
    for file in tqdm(zip_ref.infolist(), desc='Extracting files'):
      zip_ref.extract(file, 'dataset')
  os.remove('images.zip')
  shutil.move('labels.csv', 'dataset/labels.csv')

Extracting files: 100%|██████████| 500001/500001 [01:48<00:00, 4606.24it/s]


### Генерация датасета

In [ ]:
#русское слово (берем с HF)
#русское cлово + знак препинания (берем с HF и добавляем знак) ".,?!:;/%"
#русское слово с дефисом внутри слова
#число
#число, разделенное точкой или дефисом (01.01.2024)
#число + символ валюты или процент
#слово или число в скобках, кавычках (с одной стороны или с 2х) "()"«»"
#имя (И.И.)
#№ + числа
#английское слово
#электронная почта

In [ ]:
if not os.path.exists('arial-cyr.ttf') and generate_dataset:
  wget.download('https://huggingface.co/datasets/smthrgnl/arial_cyrillic_font/blob/main/arial-cyr.ttf')

Добавляем синтетический датасет

In [ ]:
def generate_image_for_word(word, output_directory, image_name):

    font_size = random.randint(50, 100)
    padding = 20
    vertical_padding = 30

    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        try:
            font = ImageFont.truetype("arialbd.ttf", font_size)
        except IOError:
            font = ImageFont.load_default()
            font.size = font_size

    temp_img = Image.new('RGB', (1, 1))
    temp_draw = ImageDraw.Draw(temp_img)

    bbox = temp_draw.textbbox((0, 0), word, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    ascent, descent = font.getmetrics()

    img_width = text_width + 2 * padding
    img_height = ascent + descent + vertical_padding

    bg_color = (random.randint(110, 255), random.randint(110, 255), random.randint(110, 255))
    text_color = (random.randint(0, 100), random.randint(0, 100), random.randint(0, 100))

    img = Image.new('RGB', (img_width, img_height), color=bg_color)
    d = ImageDraw.Draw(img)

    text_position = (padding, (img_height - text_height) // 2 - descent // 2)

    d.text(text_position, word, fill=text_color, font=font)

    image_path = os.path.join(output_directory, image_name)
    img.save(image_path, 'JPEG')

In [ ]:
r = RandomWords()

In [ ]:
punctuation = ".,?!:;/"
currency = "$"

In [ ]:
random.seed(42)

if generate_dataset:

  words_df = pd.DataFrame(columns=['filename', 'words'])
  filenames, words = [], []

  if 'images' in os.listdir('dataset'):
    shutil.rmtree('dataset/images')
  os.makedirs('dataset/images')

  with open('./dataset/1.txt', 'r', encoding='utf-8') as labels_txt:

      counter = 0

      for line in tqdm(labels_txt.readlines()):

        #if counter == 10:
        #  break

        rand = random.randint(1, 36)
        splitted_line = line.split('`')
        img_name = splitted_line[0][2:]
        img_value = splitted_line[1][:-1] if splitted_line[1][-1] == '\n' else splitted_line[1]

        if rand in range(1, 8): #обычное слово на русском языке
            shutil.copy(f'dataset/1/{img_name}', f'dataset/images/{img_name}')
        else:
          if rand in range(9, 12): #слово со знаком препинания
            img_value = img_value + random.choice(punctuation)

          elif rand in range(13, 15): #слово с дефисом
            index = random.randint(1, len(img_value) - 2)
            img_value = img_value[:index] + '-' + img_value[index:]

          elif rand in range(16, 18): #число
            img_value = ''.join(random.choices(string.digits, k=random.randint(1, 12)))

          elif rand in range(19, 21): #число с разделителями
            num = ''.join(random.choices(string.digits, k=random.randint(5, 12)))
            n = random.randint(0, 1)
            sep = '.' if n == 0 else '-'
            index1 = random.randint(1, len(num) // 2)
            index2 = random.randint(len(num) // 2 + 1, len(num) - 2)
            img_value = ''.join(num[:index1]) + sep + ''.join(num[index1:index2]) + sep + ''.join(num[index2:])

          elif rand in range(22, 24): #число + символ валюты, номер или процент
            num = ''.join(random.choices(string.digits, k=random.randint(1, 6)))
            n = random.randint(0, 2)
            if n == 0:
              suffix = '%'
              img_value = num + suffix
            elif n == 1:
              suffix = currency
              img_value = num + suffix
            else:
              img_value = '№' + num

          elif rand in range(25, 27): #объект в скобках
            n = random.randint(0, 8)
            if n == 0:
              img_value = '(' + img_value
            elif n == 1:
              img_value = img_value + ')'
            elif n == 2:
              img_value = '(' + img_value + ')'
            elif n == 3:
              img_value = '«' + img_value
            elif n == 4:
              img_value = img_value + '»'
            elif n == 5:
              img_value = '«' + img_value + '»'
            elif n == 6:
              img_value = '\"' + img_value
            elif n == 7:
              img_value = img_value + '\"'
            elif n == 8:
              img_value = '\"' + img_value + '\"'


          elif rand in range(28, 29): # инициалы (2 заглавных буквы с точками)
            img_value = '.'.join([random.choice('АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЭЮЯ') for _ in range(2)]) + '.'

          elif rand in range(30, 34): # английское слово
            eng_word = r.get_random_word()
            seed = random.randint(1, 3)
            if seed == 2:
              eng_word = eng_word[0].upper() + eng_word[1:]
            if seed == 3:
              eng_word = eng_word.upper()
            if len(eng_word) > 31:
              eng_word = eng_word[0:31]
            img_value = eng_word

          elif rand in range(35, 36): # почтовый адрес (в рандомное место вставляем _)
            mail = r.get_random_word()
            if len(mail) > 13:
              mail = mail[0:12]
            index = random.randint(1, len(mail))
            server = r.get_random_word()
            if len(server) > 13:
              server = server[0:12]
            k = random.randint(1, 3)
            if k == 3:
              mail = mail[:index] + '_' + mail[index:]
            domain = ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 3)))
            img_value = mail + '@' + server + '.' + domain

          img_name = img_name.replace('_1_', '_2_')
          fs = random.randint(22, 45)
          generate_image_for_word(word=img_value, image_name=img_name, output_directory='./dataset/images')

          if img_value[0] == '\"':
            img_value = '\\' + img_value
          if img_value[-1] == '\"':
            img_value = img_value[:-1] + '\\' + '\"'

        filenames.append(img_name)
        words.append(img_value)
        counter += 1

  words_df['filename'] = filenames
  words_df['words'] = words
  words_df.to_csv('dataset/labels.csv', sep=';', index=False, encoding='utf-8-sig')

Делим на тренировочную, тестовую, валидационную выборки

In [ ]:
def split_images(labels_file_path, image_folder_path, ranges):

    labels_df = pd.read_csv(labels_file_path, sep=';')

    for dir in ['train', 'val', 'test_']:
        images_dir = f'./{dir}/images'
        if os.path.exists(images_dir):
            shutil.rmtree(images_dir)
        os.makedirs(images_dir)

    for i, (start, end) in enumerate(ranges):

        start_index = start
        end_index = end
        if i == 0:
            path = './train'
        if i == 1:
            path = './val'
        if i == 2:
            path = './test_'

        output_file = f'{path}/labels.csv'
        labels_part = labels_df.iloc[start:end]
        labels_part.to_csv(output_file, sep=' ', index=False)
        print(f'Written lines {start} to {end} to {output_file}')

        # Copy the specified range of files
        for i in tqdm(range(start_index, end_index)):

            img_filename = labels_df['filename'].iloc[i]

            src_file = os.path.join(image_folder_path, img_filename)
            dest_file = os.path.join(f'{path}/images', img_filename)

            # Copy the file
            shutil.copy2(src_file, dest_file)

Задаем размеры

In [ ]:
labels_file = './dataset/labels.csv'
images_path = './dataset/images'
line_ranges = [(0, 30000), (30000, 40000), (40000, 50000)] #train range, val range, test range

In [ ]:
split_dataset = True
if split_dataset:
  split_images(labels_file, images_path, line_ranges)

Written lines 0 to 30000 to ./train/labels.csv


100%|██████████| 30000/30000 [00:25<00:00, 1165.77it/s]


Written lines 30000 to 40000 to ./val/labels.csv


100%|██████████| 10000/10000 [00:02<00:00, 3661.03it/s]


Written lines 40000 to 50000 to ./test_/labels.csv


100%|██████████| 10000/10000 [00:02<00:00, 3501.33it/s]


### Обучение

Клонируем библиотеку и копируем датасет в нужные директории

In [ ]:
if 'EasyOCR' not in os.listdir():
  subprocess.run(['git', 'clone', 'https://github.com/something-original/EasyOCR'])
  shutil.move('./train', './EasyOCR/trainer/all_data', copy_function=shutil.copy2)
  shutil.move('./val', './EasyOCR/trainer/all_data', copy_function=shutil.copy2)

Загружаем предобученный чекпоинт

In [ ]:
if 'cyrillic_g2.pth' not in os.listdir():
  wget.download('https://huggingface.co/smthrgnl/easy_ocr/blob/main/cyrillic_g2.pth')
if 'model' not in os.listdir('EasyOCR'):
  os.makedirs('EasyOCR/model')
  shutil.move('./cyrillic_g2.pth', 'EasyOCR/model')

Настраиваем параметры

In [ ]:
symbols = '.,?!:;/%-№()«»\$_@' + '\'' + '\"'

In [ ]:
params = {
  'number': '0123456789',
  'symbol': symbols,
  'lang_char': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя',
  'experiment_name': 'fmd',
  'train_data': 'all_data/train',
  'valid_data': 'all_data/val',
  'manualSeed': 1111,
  'workers': 6,
  'batch_size': 64, #32
  'num_iter': 30000,
  'valInterval': 200,
  'saved_model': 'cyrillic_g2',
  'FT': True,
  'optim': False, # значение по умолчанию - Adadelta
  'lr': 1.,
  'beta1': 0.9,
  'rho': 0.95,
  'eps': 0.00000001,
  'grad_clip': 5,

  'select_data': 'train', # это папка dataset в train_data
  'batch_ratio': '1',
  'total_data_usage_ratio': 1.0,
  'batch_max_length': 68,
  'imgH': 64,
  'imgW': 600,
  'rgb': False,
  'contrast_adjust': False,
  'sensitive': True,
  'PAD': True,
  'contrast_adjust': 0.0,
  'data_filtering_off': False,

  'Transformation': 'None',
  'FeatureExtraction': 'VGG',
  'SequenceModeling': 'BiLSTM',
  'Prediction': 'CTC',
  'num_fiducial': 20,
  'input_channel': 1,
  'output_channel': 256,
  'hidden_size': 256,
  'decode': 'greedy',
  'new_prediction': False,
  'freeze_FeatureFxtraction': False,
  'freeze_SequenceModeling': False
}

In [ ]:
with open('EasyOCR/trainer/config_files/custom_data_train.yaml', 'w') as file:
    yaml.dump(params, file)

#### Запуск обучения

In [ ]:
!pip install -r EasyOCR/requirements.txt

In [ ]:
import sys

sys.path.insert(0, './EasyOCR/trainer')

In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

cudnn.benchmark = True
cudnn.deterministic = False

def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt


In [ ]:
opt

{'FT': True,
 'FeatureExtraction': 'VGG',
 'PAD': True,
 'Prediction': 'CTC',
 'SequenceModeling': 'BiLSTM',
 'Transformation': 'None',
 'batch_max_length': 68,
 'batch_ratio': ['1'],
 'batch_size': 64,
 'beta1': 0.9,
 'contrast_adjust': 0.0,
 'data_filtering_off': False,
 'decode': 'greedy',
 'eps': 1e-08,
 'experiment_name': 'fmd',
 'freeze_FeatureFxtraction': False,
 'freeze_SequenceModeling': False,
 'grad_clip': 5,
 'hidden_size': 256,
 'imgH': 64,
 'imgW': 600,
 'input_channel': 1,
 'lang_char': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя',
 'lr': 1.0,
 'manualSeed': 1111,
 'new_prediction': False,
 'num_fiducial': 20,
 'num_iter': 30000,
 'number': '0123456789',
 'optim': False,
 'output_channel': 256,
 'rgb': False,
 'rho': 0.95,
 'saved_model': 'cyrillic_g2',
 'select_data': ['train'],
 'sensitive': True,
 'symbol': '.,?!:;/%-№()«»\\$_@\'"',
 'total_data_usage_ratio': 1.0,
 'train_data': 'all_data/trai

In [ ]:
#Запускаем обучение
opt = get_config("EasyOCR/trainer/config_files/custom_data_train.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/train
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/train	 dataset: train


AssertionError: datasets should not be an empty iterable